In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sympy import (symbols, simplify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody, angular_momentum

from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

init_vprinting()

### Robotics Toolbox Model

In [ ]:
from roboticstoolbox.models.DH import Panda, Puma560

In [ ]:
panda = Panda()
panda2 = Panda()

In [ ]:
panda

In [ ]:
6.1597-10.7

In [ ]:
r_arr = np.array([
    [3.875e-03, 2.081e-03, 0],
    [-3.141e-03, -2.872e-02, 3.495e-03],
    [2.7518e-02, 3.9252e-02, -6.6502e-02],
    [-5.317e-02, 1.04419e-01, 2.7454e-02],
    [-1.1953e-02, 4.1065e-02, -3.8437e-02],
    [6.0149e-02, -1.4117e-02, -1.0517e-02],
    [1.0517e-02, -4.252e-03, -4.5403e-02],
])

In [ ]:
for ii, link in enumerate(panda.links):
    link.r = r_arr[ii,:]

In [ ]:
def tau_from_state(q, qd, qdd):
    panda = Panda()
    M = panda.inertia(q)
    C = panda.coriolis(q, qd)
    G = panda.gravload(q)
    tau = M@qdd + C@qd + G
    return tau

In [ ]:
def randq():
    panda = Panda()
    qlim = np.zeros([7,2])
    for ii, link in enumerate(panda.links):
        qlim[ii] = link.qlim
    #qlim
    r = np.random.random_sample(7)
    d = qlim[:,1]-qlim[:,0]
    c = qlim[:,0]
    return c + d*r
    

In [ ]:
def randpath(N):
    path = np.zeros([N,7])
    for ii in range(N):
        path[ii,:] = randq()
    return np.round(path, 3)

In [ ]:
def rand_traj(N):
    traj = np.zeros([N, 15])
    traj[:,:7] = randpath(N)
    traj[1:-1, 7:14] = np.round((traj[2:,:7] - traj[:-2,:7])/2, 3)
    traj[:,-1] = np.arange(N)
    return traj

In [ ]:
from optibot.schemes import extend_array

In [ ]:
def savetraj(N, fname = 'ejtraj.txt'):
    traj = extend_array(rand_traj(N))
    traj[-1, -1] += 0.2
    np.savetxt(fname, traj, '%1.3f')
    return traj

def loadtraj(fname = 'ejtraj.txt'):
    return np.loadtxt(fname)

def savecustomtraj(traj, fname = 'ejtraj.txt'):
    traj[-1, -1] += 0.2
    np.savetxt(fname, traj, '%1.3f')

### Generating, saving and loading a random trajectory

In [ ]:
savetraj(7)
ej_traj = loadtraj()

In [ ]:
from scipy.interpolate import CubicHermiteSpline as hermite

In [ ]:
her_traj = hermite(ej_traj[:,-1], ej_traj[:, :7], ej_traj[:, 7:14])
her_speed = her_traj.derivative()
her_accel = her_speed.derivative()
interp_f = [her_traj, her_speed, her_accel]

In [ ]:
plt.style.use('default')

plt.style.use('default')
t_plot = np.linspace(0, ej_traj[-1, -1], 500)
for ii, varname in enumerate(['q', 'q_dot']):
    plt.figure(figsize=[12,8])
    plt.title('ideal ' + varname)
    plt.plot(ej_traj[:,-1], ej_traj[:, 7*ii: 7*(ii+1)], 'o')
    plt.plot(t_plot, interp_f[ii](t_plot))
    plt.grid()

interp_n = 500
t_plot = np.linspace(0, ej_traj[-1, -1], interp_n)
tau_arr = np.zeros([interp_n, 7])
for ii in range(interp_n):
    _t = t_plot[ii]
    _q = her_traj(_t)
    _qd = her_speed(_t)
    _qdd = her_accel(_t)
    _tau = tau_from_state(_q, _qd, _qdd)
    tau_arr[ii,:] = _tau

plt.figure(figsize=[12,8])
plt.title('ideal torque')
plt.plot(t_plot, tau_arr)
plt.grid()

N = 22
_ctraj = np.zeros([N,15])
_ctraj[:,6] = 1
_ctraj[:,5] = 0.4
_ctraj[:,-1] = np.arange(N)*3

_ctraj[0:6, 3] = -0.2
_ctraj[0:6, 1] = -0.2
_ctraj[2:4, 5] = np.arctan(0.107/0.088)
_ctraj[4:6, 5] = np.arctan(0.107/0.088)+np.pi/2

_ctraj[6:18, 3] = -np.pi/2
_ctraj[8:10, 5] = np.arctan(0.107/0.088)
_ctraj[10:12, 5] = np.arctan(0.107/0.088)+np.pi/2
_ctraj[12:18, 4] = np.pi/2
_ctraj[14:16, 5] = np.arctan(0.107/0.088)
_ctraj[16:18, 5] = np.arctan(0.107/0.088)+np.pi/2

_ctraj[18:22, 1] = -np.pi/2
_ctraj[18:22, 3] = -np.pi/2
_ctraj[20:22, 2] = -np.pi/2
_ctraj[:,:7]

In [ ]:
th5_0 = 0.4
th5_1 = 3.6
dth = 0.2
dt = 1
t_change = 3

N_th = 2*int(1+(th5_1 - th5_0)/dth)
N_change = int(t_change/dt)
N_th_2 = N_th+ N_change
th_arr = np.linspace(th5_0, th5_1, int(N_th/2))

N = 2*N_th + N_change
_ctraj = np.zeros([N,15])
_ctraj[:,6] = 1
_ctraj[:,5] = th5_0
_ctraj[:,-1] = np.arange(N)*dt

_ctraj[0:N_th, 3] = -0.2
_ctraj[0:N_th, 1] = -0.2

_ctraj[0:N_th:2, 5] = th_arr
_ctraj[1:N_th:2, 5] = th_arr



_ctraj[N_th_2:, 1] = 0
_ctraj[N_th_2:, 3] = -np.pi/2

_ctraj[N_th_2::2, 5] = th_arr
_ctraj[N_th_2+1::2, 5] = th_arr



her_traj = hermite(_ctraj[(N_th-1, N_th_2),-1], _ctraj[(N_th-1, N_th_2), :7], _ctraj[(N_th-1, N_th_2), 7:14])
her_speed = her_traj.derivative()
_ctraj[N_th: N_th_2, :7] = her_traj(_ctraj[N_th: N_th_2,-1])
_ctraj[N_th: N_th_2, 7:14] = her_speed(_ctraj[N_th: N_th_2,-1])


_ctraj[:,:7]

In [ ]:
_ctraj[-1,-1]

In [ ]:
savecustomtraj(_ctraj, 'custom_trajectory.txt')

In [ ]:
# import the data
file_path = 'log_custom_trajectory.txt'
d = np.loadtxt(file_path)
#t,knot_p = d[:,0],d[:,64]
q_cub,q_lin,q,q_d,dq,dq_d,tau_j,tau_jd, gvec = [d[:,i*7+1:i*7+8] for i in range(9)]

In [ ]:
t = np.linspace(0, _ctraj[-1,-1], q.shape[0])

In [ ]:
q.shape

In [ ]:
d

In [ ]:
def t_index(segment):
    t = segment*dt
    t_c = q.shape[0]/_ctraj[-1,-1]
    
    return int(t*t_c)

In [ ]:
_ctraj.shape

In [ ]:
t_index(30)

In [ ]:
tau_mean = tau_j.copy()
tau_mean_simp = np.zeros([_ctraj.shape[0], tau_j.shape[1]])
for ii in range(tau_j.shape[1]):
    for kk in range(_ctraj.shape[0]-1):
        s0 = t_index(kk)
        s1 = t_index(kk+1)
        tau_mean[s0:s1,ii] = np.mean(tau_j[s0:s1,ii])
        tau_mean_simp[kk,ii] = np.mean(tau_j[s0:s1,ii])

In [ ]:
s1 = 0
s2 = 10
ii = 5
plt.figure(figsize=[15,10])
plt.plot(t[t_index(s1):t_index(s2)],tau_jd[t_index(s1):t_index(s2),ii])
plt.plot(t[t_index(s1):t_index(s2)],tau_j[t_index(s1):t_index(s2),ii])
plt.plot(t[t_index(s1):t_index(s2)],tau_mean[t_index(s1):t_index(s2),ii])
#plt.plot(t[t_index(s1):t_index(s2)],q[t_index(s1):t_index(s2),ii])
plt.plot(t[t_index(s1):t_index(s2)],gvec[t_index(s1):t_index(s2),ii])
plt.grid()

In [ ]:
tau_mean_simp

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def _torque_vert(theta, beta, Me):
    return Me*np.cos(theta-beta)

In [ ]:
def _torque_vert_adv(theta, l0, l1, m):
    return m*9.8*(l0*np.cos(theta) + l1*np.sin(theta))

In [ ]:
s0 = 0
s1 = N_th
[beta_s, Me_s], _ = curve_fit(
    _torque_vert,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,5],
    (0.66, 2.9),
    bounds=([-np.pi/2,0],[np.pi/2, 10])
)
beta_s, Me_s

In [ ]:
s0 = 0
s1 = N_th
[l0_s, l1_s, m_s], _ = curve_fit(
    _torque_vert_adv,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,5],
    (0.075, 0.052, 3.14),
    bounds=([0,0,3.14],[0.088,0.11, 3.1401])
)
l0_s, l1_s, m_s

In [ ]:
s0 = 0
s1 = N_th
[l0_e, l1_e, m_e], _ = curve_fit(
    _torque_vert_adv,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,5],
    (0.075, 0.052, 3.34),
    bounds=([0,0,3.34],[0.088,0.11, 3.3401])
)
l0_e, l1_e, m_e

In [ ]:
l0_m = 0.076
l1_m = 0.058

l0_m2 = 0.072
l1_m2 = 0.054

l0_m3 = 0.075
l1_m3 = 0.057

In [ ]:
ii = 5
s0 = 0
s1 = N_th
dumm = np.arange(0,30,1)
plt.figure(figsize=[12,10])
plt.plot(_ctraj[s0:s1:2,ii],tau_mean_simp[s0:s1:2,ii], 'o')
print(dumm[s0:s1:2])
n_interp = 200
theta_arr = np.linspace(_ctraj[s0,ii], _ctraj[s1-1,ii], n_interp)

Me = 2.9
beta = 0.66
#plt.plot(theta_arr, Me*np.cos(theta_arr-beta))
#plt.plot(theta_arr, _torque_vert(theta_arr, beta_s, Me_s))
l0=0.075
l1=0.052
m = 3.14

plt.plot(theta_arr, _torque_vert_adv(theta_arr, l0_s, l1_s, 3.14),label = 'fit parameters')
plt.plot(theta_arr, _torque_vert_adv(theta_arr, l0_m, l1_m, 3.14), label = 'manual parameters A')
plt.plot(theta_arr, _torque_vert_adv(theta_arr, l0_m3, l1_m3, 3.14), label = 'manual parameters B')
plt.grid()
plt.legend()

In [ ]:
beta_s, Me_s

In [ ]:
def _torque_horz(theta, beta, Me):
    return -Me*np.sin(beta-theta)

In [ ]:
def _torque_horz_adv(theta, l0, l1, m):
    return m*9.8*(l0*np.sin(theta) - l1*np.cos(theta))

In [ ]:
s0 = N_th_2
s1 = N
[beta_s2, Me_s2], _ = curve_fit(
    _torque_horz,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,5],
    (0.66, 2.9),
    bounds=([-np.pi/2,0],[np.pi/2, 10])
)
beta_s2, Me_s2

In [ ]:
s0 = N_th_2
s1 = N
[l0_s2, l1_s2, m_s2], _ = curve_fit(
    _torque_horz_adv,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,5],
    (0.075, 0.052, 3.14),
    bounds=([0,0,3],[0.088,0.11, 3.1401])
)
l0_s2, l1_s2, m_s2

In [ ]:
s0 = N_th_2
s1 = N
[l0_e2, l1_e2, m_e2], _ = curve_fit(
    _torque_horz_adv,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,5],
    (0.075, 0.052, 3.34),
    bounds=([0,0,3],[0.088,0.11, 3.3401])
)
l0_e2, l1_e2, m_e2

In [ ]:
l0_s, l1_s, m_s

In [ ]:
ii = 5
s0 = N_th_2
s1 = N
dumm = np.arange(0,30,1)
plt.figure(figsize=[12,10])
plt.plot(_ctraj[s0:s1:2,ii],tau_mean_simp[s0:s1:2,ii], 'o')
#print(dumm[s0:s1:2])
n_interp = 200
theta_arr = np.linspace(_ctraj[s0,ii], _ctraj[s1-1,ii], n_interp)

Me = 3.01
beta = 0.65
#plt.plot(theta_arr, -Me_s2*np.sin(beta_s2-theta_arr))
plt.plot(theta_arr, _torque_horz_adv(theta_arr, l0_s2, l1_s2, 3.14), label = 'fit parameters')
plt.plot(theta_arr, _torque_horz_adv(theta_arr, l0_m, l1_m, 3.14), label = 'manual parameters A')
plt.plot(theta_arr, _torque_horz_adv(theta_arr, l0_m3, l1_m3, 3.14), label = 'manual parameters B')
plt.grid()
plt.legend()

In [ ]:
beta_s2, Me_s2, beta_s, Me_s

In [ ]:
def _torque_shoulder_horz(theta, M3, m, Me, beta):
    w = m*9.8
    return M3 + w*0.384 - Me*np.sin(beta-theta)

In [ ]:
def _torque_shoulder_horz_adv(theta, M3, l0, l1):
    return M3 + 3.14*9.8*(0.384 + l0*np.sin(theta) - l1*np.cos(theta))

In [ ]:
def _torque_shoulder_horz_simp(theta, M3):
    return M3 + 3.14*9.8*(0.384 + l0_m*np.sin(theta) - l1_m*np.cos(theta))
def _torque_shoulder_horz_simp2(theta, M3):
    return M3 + 3.34*9.8*(0.384 + l0_m2*np.sin(theta) - l1_m2*np.cos(theta))
def _torque_shoulder_horz_simp3(theta, M3):
    return M3 + 3.14*9.8*(0.384 + l0_m3*np.sin(theta) - l1_m3*np.cos(theta))

In [ ]:
s0 = N_th_2
s1 = N

Me_test = Me_s2
beta_test = beta_s2

[M3_s, m_s, Me_s3, beta_s3], _ = curve_fit(
    _torque_shoulder_horz,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,3],
    (16.8, 0.8, Me_test, beta_test),
    bounds=([0,0, Me_test*0.5, beta_test*0.5],[20, 3, Me_test, beta_test])
)
M3_s, m_s, Me_s3, beta_s3

In [ ]:
s0 = N_th_2
s1 = N
[M3_s, l0_s3, l1_s3], _ = curve_fit(
    _torque_shoulder_horz_adv,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,3],
    (5.13, 0.075, 0.052),
    bounds=([0,0,0],[9,0.088,0.11])
)
M3_s, l0_s3, l1_s3

In [ ]:
s0 = N_th_2
s1 = N
[M3_s2,], _ = curve_fit(
    _torque_shoulder_horz_simp,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,3],
    (5.13),
    bounds=([0,],[9,])
)
M3_s2

In [ ]:
s0 = N_th_2
s1 = N
[M3_s3,], _ = curve_fit(
    _torque_shoulder_horz_simp2,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,3],
    (5.13),
    bounds=([0,],[9,])
)
M3_s3

In [ ]:
s0 = N_th_2
s1 = N
[M3_s4,], _ = curve_fit(
    _torque_shoulder_horz_simp3,
    _ctraj[s0:s1:2,5],
    tau_mean_simp[s0:s1:2,3],
    (5.13),
    bounds=([0,],[9,])
)
M3_s4

In [ ]:
print(l0_s, l1_s)
print(l0_s2, l1_s2)
print(l0_s3, l1_s3)

In [ ]:
ii = 3
s0 = N_th_2
s1 = N
#dumm = np.arange(0,30,1)
plt.figure(figsize=[12,10])
plt.plot(_ctraj[s0:s1:2,5],tau_mean_simp[s0:s1:2,ii], 'o')
#print(dumm[s0:s1:2])
n_interp = 200
theta_arr = np.linspace(_ctraj[s0,5], _ctraj[s1-1,5], n_interp)

M3 = 16.8
Me = 2.9
m = 0.8
w = m*9.8
l = Me/w
beta = 0.66

plt.plot(theta_arr, _torque_shoulder_horz_adv(theta_arr, M3_s, l0_s3, l1_s3), label = 'fit parameters')
plt.plot(theta_arr, _torque_shoulder_horz_simp(theta_arr, M3_s2), label = 'manual parameters A')
plt.plot(theta_arr, _torque_shoulder_horz_simp3(theta_arr, M3_s4), label = 'manual parameters B')
#_x = tau_mean_simp[s0:s1:2,ii] - w*0.384 + Me*np.sin(beta-_ctraj[s0:s1:2,5])
#plt.plot(_ctraj[s0:s1:2,5],_x-np.min(_x), 'o')
#plt.vlines(beta, 19.5,23, color='k', ls = ':')
plt.grid()
plt.legend()

In [ ]:
9.8*(3.6*0.06+1.23*0.25)

In [ ]:
9.8*(3.6*0.12+1.23*0.384)

In [ ]:
print(panda.links[3].m, panda.links[4].m)

In [ ]:
n_interp = 200

Me = np.mean([Me_s, Me_s2, Me_s3])
beta = np.mean([beta_s, beta_s2, beta_s3])
m = m_s
M3 = M3_s

s0 = 0
s1 = N_th
theta_arr = np.linspace(_ctraj[s0,5], _ctraj[s1-1,5], n_interp)
plt.figure(figsize=[12,10])
plt.plot(_ctraj[s0:s1:2,5],tau_mean_simp[s0:s1:2,5], 'o')
plt.plot(theta_arr, _torque_vert(theta_arr, beta, Me))
plt.grid()


s0 = N_th_2
s1 = N
theta_arr = np.linspace(_ctraj[s0,5], _ctraj[s1-1,5], n_interp)

plt.figure(figsize=[12,10])
plt.plot(_ctraj[s0:s1:2,5],tau_mean_simp[s0:s1:2,5], 'o')
plt.plot(theta_arr, -Me*np.sin(beta-theta_arr))
plt.grid()


plt.figure(figsize=[12,10])
plt.plot(_ctraj[s0:s1:2,5],tau_mean_simp[s0:s1:2,3], 'o')
plt.plot(theta_arr, _torque_shoulder_horz(theta_arr, M3, m, Me, beta))
plt.grid()

M3, m, Me, beta, Me/m/9.81

In [ ]:
th5_0 = 0.4
th5_1 = 3.6
dth = 0.2
dt = 1
t_change = 3

N_th = 2*int(1+(th5_1 - th5_0)/dth)
N_change = int(t_change/dt)
N_th_2 = N_th+ N_change
th_arr = np.linspace(th5_0, th5_1, int(N_th/2))

N = 2*N_th + N_change
_ctraj = np.zeros([N,15])
_ctraj[:,6] = 1
_ctraj[:,5] = th5_0
_ctraj[:,-1] = np.arange(N)*dt

_ctraj[0:N_th, 3] = -0.2
_ctraj[0:N_th, 1] = -0.2

_ctraj[0:N_th:2, 5] = th_arr
_ctraj[1:N_th:2, 5] = th_arr



_ctraj[N_th_2:, 1] = 0
_ctraj[N_th_2:, 3] = -np.pi/2

_ctraj[N_th_2::2, 5] = th_arr
_ctraj[N_th_2+1::2, 5] = th_arr



her_traj = hermite(_ctraj[(N_th-1, N_th_2),-1], _ctraj[(N_th-1, N_th_2), :7], _ctraj[(N_th-1, N_th_2), 7:14])
her_speed = her_traj.derivative()
_ctraj[N_th: N_th_2, :7] = her_traj(_ctraj[N_th: N_th_2,-1])
_ctraj[N_th: N_th_2, 7:14] = her_speed(_ctraj[N_th: N_th_2,-1])


_ctraj[:,:7]

# HERE BE DRAGONS

(copypasted and untested cells, do not use beyond this point)

In [ ]:
for link in panda.links:
    print(link.m)

In [ ]:
from sympy import (symbols, simplify, lambdify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody, angular_momentum

In [ ]:
from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

In [ ]:
init_vprinting()

In [ ]:
panda.qlim

In [ ]:
link.r


In [ ]:
def matrix_to_6_vect(M):
    I = np.zeros(6)
    I[0] = M[0,0]
    I[1] = M[1,1]
    I[2] = M[2,2]
    I[3] = M[0,1]
    I[4] = M[1,2]
    I[5] = M[0,2]
    return I

def arr_to_vect(v, frame):
    return v[0]*frame.x + v[1]*frame.y + v[2]*frame.z

def make_search_func(link, replacedict):
    def search(varname):
        if varname in replacedict.keys():
            return replacedict[varname]
        else:
            return getattr(link, varname)
    return search

In [ ]:
N_in = ReferenceFrame('N')
P0 = Point('P0')
P0.set_vel(N_in, 0)

def dhlink_to_symbody(dhlink, ref_frame, ref_point, N_inert, P_inert, q, name = 'Body', g = 9.8, replacedict = {}):
    search = make_search_func(dhlink, replacedict)
    theta = search('theta')
    alpha = search('alpha')
    d = search('d')
    a = search('a')
    m = search('m')
    inert = matrix_to_6_vect(search('I'))
    
    
    Nint = ref_frame.orientnew('Nint', 'Axis', [theta, ref_frame.z])
    N1 = Nint.orientnew('N1', 'Axis', [alpha, Nint.x])
    Nbody = N1.orientnew('NB', 'Axis', [q, N1.z])
    P1 = ref_point.locatenew('P1', d* ref_frame.z + a * N1.x)
    P1.set_vel(N_in, P1.pos_from(P_inert).dt(N_inert))
    
    
    cm_pos = arr_to_vect(search('r'), Nbody)
    CM0 = ref_point.locatenew('CM0', cm_pos)
    CM0.set_vel(N_in, CM0.pos_from(P_inert).dt(N_inert))
    I_0 = inertia(Nbody, *inert)
    body0 = RigidBody(name, CM0, Nbody, m, (I_0,CM0))
    body0.potential_energy = m * g * CM0.pos_from(P_inert).dot(N_inert.z)
    
    return body0, Nbody, P1

In [ ]:
def dhlink_mod_to_symbody(dhlink, ref_frame, ref_point, N_inert, P_inert, q, name = 'Body', g = 9.8, replacedict = {}):
    search = make_search_func(dhlink, replacedict)
    theta = search('theta')
    alpha = search('alpha')
    d = search('d')
    a = search('a')
    m = search('m')
    inert = matrix_to_6_vect(search('I'))
    
    
    Nint = ref_frame.orientnew('Nint', 'Axis', [alpha, ref_frame.x])
    N1 = Nint.orientnew('N1', 'Axis', [theta, Nint.z])
    Nbody = N1.orientnew('NB', 'Axis', [q, N1.z])
    P1 = ref_point.locatenew('P1', d* N1.z + a * ref_frame.x)
    P1.set_vel(N_inert, P1.pos_from(P_inert).dt(N_inert))
    
    
    cm_pos = arr_to_vect(search('r'), Nbody)
    #CM0 = ref_point.locatenew('CM0', cm_pos)
    CM0 = P1.locatenew('CM0', cm_pos)
    CM0.set_vel(N_in, CM0.pos_from(P_inert).dt(N_inert))
    I_0 = inertia(Nbody, *inert)
    body0 = RigidBody(name, CM0, Nbody, m, (I_0,CM0))
    body0.potential_energy = m * g * CM0.pos_from(P_inert).dot(N_inert.z)
    
    return body0, Nbody, P1

In [ ]:
_b1, _N1, _P1 = dhlink_mod_to_symbody(link, N_in, P0, N_in, P0, dynamicsymbols('q'))

In [ ]:
_b1.kinetic_energy(N_in)

In [ ]:
from sympy import pi

In [ ]:
hpi = pi/2
alpha_arr = [0, -hpi, hpi, hpi, -hpi, hpi, hpi]

points= [P0]
frames = [N_in]
bodys = []
for ii,link in enumerate(panda.links):
    _b1, _N1, _P1 = dhlink_mod_to_symbody(
        link, frames[-1], points[-1], N_in, P0, dynamicsymbols(f'q_{ii}'),
        replacedict={'alpha':alpha_arr[ii]}
    )
    points.append(_P1)
    frames.append(_N1)
    bodys.append(_b1)

In [ ]:
bodys[0].frame

In [ ]:
Lag = Lagrangian(N_in, *bodys)

Lag_simp

In [ ]:
panda.mdh

In [ ]:
sym_x = dynamicsymbols('q_0:7')
sym_x = sym_x + [ii.diff() for ii in sym_x]
sym_params = []

In [ ]:
_p = points[7].pos_from(P0)
_p.express(N_in).subs({qq:0 for qq in sym_x})

In [ ]:
for link in panda.links:
    print(link.alpha)

In [ ]:
link.mdh

In [ ]:
len(panda.links)

In [ ]:
from optibot.symbolic import ImplicitLagrangesMethod

In [ ]:
u0, u1, u2, u3, u4, u5, u6 = symbols('u_:7')
qq = dynamicsymbols('q_0:7')
uu = [u0, u1, u2, u3, u4, u5, u6]
FL = []
for ii in range(7):
    fr = bodys[ii].frame
    FL.append((fr, uu[ii]*fr.z))
for ii in range(6):
    fr = bodys[ii].frame
    FL.append((fr, -uu[ii+1]*bodys[ii+1].frame.z))
    
LM_small = ImplicitLagrangesMethod(Lag, qq, forcelist=FL, frame=N_in)

In [ ]:
try:
    from jbkjhbkhbkhb import gg
except ModuleNotFoundError:
    print('te cogí')
else:
    print('ok')

In [ ]:
_ = LM_small.form_lagranges_equations()
mass_matrix_np = lambdify([sym_x, sym_params], LM_small.mass_matrix,'numpy')

In [ ]:
sym_u = uu
F_impl_np = lambdify([sym_x, sym_u, sym_params], LM_small.forcing,'numpy')

In [ ]:
_q = randq()
_qd = randq()
_x = np.concatenate((_q,_qd))

In [ ]:
_m = mass_matrix_np(_x,[])
plt.figure(figsize=[10,10])
plt.imshow(_m)
plt.colorbar()

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(panda.inertia(_q))
plt.colorbar()

In [ ]:
_m/panda.inertia(_q)

In [ ]:
_m

In [ ]:
from optibot.analysis import generate_G

In [ ]:
F_impl_np = lambdify([sym_x, uu, sym_params], LM_small.forcing,'numpy')

In [ ]:
G_nump = generate_G(F_impl = F_impl_np,
           M = mass_matrix_np,)

In [ ]:
G_nump(np.zeros(14), np.zeros(7), [])

In [ ]:
panda.accel(np.zeros(7), np.zeros(7), np.zeros(7))

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
%matplotlib inline

In [ ]:
_p = points[7].pos_from(P0)
_p.express(N_in).subs({qq:0 for qq in sym_x})

In [ ]:
_p.dot(N_in.z)

In [ ]:
def get_line_f():
    versors = [N_in.x, N_in.y, N_in.z]
    functions = {}
    for coord in range(3):
        for ii in range(7):
            point = points[ii+1].pos_from(P0)
            sym_leng = point.dot(versors[coord])
            key = f'{coord}{ii}'
            f = lambdify([qq,], sym_leng)
            functions[key] = f
    def line3d(q):
        xline = np.zeros(8)
        yline = np.zeros(8)
        zline = np.zeros(8)
        lines = [xline, yline, zline]
        for coord in range(3):
            for ii in range(7):
                key = f'{coord}{ii}'
                f = functions[key]
                lines[coord][ii+1] = f(q)
        return lines
    return line3d
        

In [ ]:
line3d = get_line_f()

In [ ]:
line3d(np.zeros(7))

In [ ]:
def set_axes_equal(ax: plt.Axes):
    """Set 3D plot axes to equal scale.

    Make axes of 3D plot have equal scale so that spheres appear as
    spheres and cubes as cubes.  Required since `ax.axis('equal')`
    and `ax.set_aspect('equal')` don't work on 3D.
    """
    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])
    origin = np.mean(limits, axis=1)
    radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
    _set_axes_radius(ax, origin, radius)

def _set_axes_radius(ax, origin, radius):
    x, y, z = origin
    ax.set_xlim3d([x - radius, x + radius])
    ax.set_ylim3d([y - radius, y + radius])
    ax.set_zlim3d([z - radius, z + radius])
def plot3d(q):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    xline, yline, zline = line3d(q)
    ax.plot3D(xline, yline, zline, 'red', lw =5)
    ax.set_box_aspect([1,1,1])
    set_axes_equal(ax)

In [ ]:
_q = randq()

In [ ]:
_q0 = np.zeros(7)
_q0[-2] = np.arctan(0.107/0.088)+np.pi/2

In [ ]:
plot3d(_q0)

In [ ]:
panda.plot(_ctraj[8,:7])

In [ ]:
%matplotlib notebook
%config NotebookBackend.figure_format = 'retina'

In [ ]:
panda.links[1].friction

### Casadi optimization

In [ ]:
from optibot.opti import Opti_Problem
import time
from functools import lru_cache

In [ ]:
_opti = Opti_Problem(
    LM= lambda q, qd, u, p:panda.accel(q, qd, u),
    params = [],
    verbose=True,
    silent=False
    
)

In [ ]:
_opti.dynamic_setup('g_q', 7, 7,)

In [ ]:
@lru_cache(maxsize=None)
def five_link(scheme, N, ini_guess='lin', solve_repetitions = 1, t_end = 0.7, step_length = 0.5, verbose = False, silent = True):
    _opti = Opti_Problem(
        LM=LM_small,
        params = params,
        scheme = scheme,
        ini_guess= ini_guess,
        t_end = t_end,
        verbose=verbose,
        silent=silent,
    )
    _opti.dynamic_setup()
    _opti.opti_setup(N)
    _guess = [-0.3, 0.7, 0, -0.5, -0.6]
    _opti.initial_guess(_guess, _guess[::-1])
    _opti.u_sq_cost()
    _opti.apply_scheme()
    opti = _opti.opti
    X = _opti.opti_arrs['x']
    U = _opti.opti_arrs['u']
    X_s = _opti.opti_points['x_s']
    X_e = _opti.opti_points['x_e']
    N = _opti.N
    #Periodic gait constraint:
    opti.subject_to(simetric_5_links(X_s, X_e) == 0)
    opti.subject_to(impact_cond_cas_f(X_e, X_s, params, additional_params) == 0)
    #Step size constraint:
    opti.subject_to(feet_x_cas(X[-1,:], params, additional_params) == step_length)
    opti.subject_to(feet_y_cas(X[-1,:], params, additional_params) == 0)
    #Small Feet Conditions:
    opti.subject_to(U[:,0] == 0)
    if 'parab' in scheme:
        U_c = _opti.opti_arrs['u_c']
        opti.subject_to(U_c[:,0] == 0)
    opti.subject_to(feet_y_vel_cas(X_s, params, additional_params)>0)
    opti.subject_to(feet_y_vel_cas(X_e, params, additional_params)<0)
    #Feet over ground Restrictions:
    for ii in range(1,N):
        opti.subject_to(feet_y_cas(X[ii,:], params, additional_params) > 0)
    _opti.chrono_solve(solve_repetitions=solve_repetitions,)
    return _opti.results

In [ ]:
_res = five_link('LGL', 16, 'lin', silent=False, verbose=True)

from optibot.casadi import accelrestriction2casadi
from optibot.schemes import (euler_accel_restr, trapz_accel_restr, trapz_mod_accel_restr,
                             hs_mod_accel_restr, hs_accel_restr, hs_half_x)

In [ ]:
_res

In [ ]:
plt.figure(figsize=[14,10])
labels= ['stance anckle', 'stance knee', 'stance hip', 'swing hip', 'swing knee']
for ii in range(5):
    plt.plot(_res['t'],_res['q'][:,ii], marker = 'o', label = labels[ii])
#    plt.plot(_res['t_c'],_res['q_c'][:,ii], 'o', label = labels[ii] + ' u_c')
plt.grid()
plt.legend()

plt.figure(figsize=[14,10])
labels= ['stance tibia', 'stance femur', 'torso', 'swing femur', 'swing tibia']
for ii in range(5):
    plt.plot(T_sol_arr, X_sol[:,ii], marker = 'o', label = labels[ii])
    plt.plot(T_c_arr,X_c_sol[:,ii], 'o', label = labels[ii] + ' x_c')
plt.grid()
plt.legend()

In [ ]:
def chain_to_draw(x,params):
    [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
    ] = params
    points_x = [0, ]
    points_y = [0, ]
    points_x.append(points_x[-1] - l_0_n*np.sin(x[0]))
    points_x.append(points_x[-1] - l_1_n*np.sin(x[1]))
    points_x.append(points_x[-1] - l_2_n*np.sin(x[2]))
    points_x.append(points_x[-2])
    points_x.append(points_x[-1] + l_3_n*np.sin(x[3]))
    points_x.append(points_x[-1] + l_4_n*np.sin(x[4]))
    
    
    points_y.append(points_y[-1] + l_0_n*np.cos(x[0]))
    points_y.append(points_y[-1] + l_1_n*np.cos(x[1]))
    points_y.append(points_y[-1] + l_2_n*np.cos(x[2]))
    points_y.append(points_y[-2])
    points_y.append(points_y[-1] - l_3_n*np.cos(x[3]))
    points_y.append(points_y[-1] - l_4_n*np.cos(x[4]))
    
    return points_x, points_y

In [ ]:
plt.figure(figsize=[15,15])
plt.grid()
for ii in range(0, 1, 1):
    points_x, points_y = chain_to_draw(_res['x'][ii], params)
    plt.plot(points_x, points_y, marker = 'o', lw=4, ms=12, color = plt.cm.viridis(ii/1))
    

plt.gca().set_aspect('equal')

total_mass = m_0_n + m_1_n + m_2_n + m_3_n + m_4_n
ang_mom_arr = [ang_mom_p0_np(X_sol[ii,:],params) for ii in range(N+1)]
ang_mom_swing_foot_arr = [ang_mom_p5_np(X_sol[ii,:],params, additional_params) for ii in range(N+1)]
ang_mom_swing_foot_static_arr = [ang_mom_p5_static_np(X_sol[ii,:],params, additional_params) for ii in range(N+1)]
cm_torque_arr = [total_mass * -g_n * sys_cm_np(X_sol[ii,:], params)[0] for ii in range(N+1)]
ang_mom_arr_deriv = np.gradient(ang_mom_arr, T_sol_arr)

plt.figure(figsize=[15,10])

plt.plot(T_sol_arr, ang_mom_arr, label = 'angular moment around stance foot')
plt.plot(T_sol_arr, ang_mom_swing_foot_arr, label = 'angular moment around swing foot')
plt.plot(T_sol_arr, ang_mom_swing_foot_static_arr, label = 'angular moment around static point at swing foot')
plt.plot(T_sol_arr, cm_torque_arr)
plt.plot(T_sol_arr, ang_mom_arr_deriv)
plt.grid()
plt.legend()

system_energy_arr = [system_energy_np(X_sol[ii,:],params) for ii in range(N+1)]
system_energy_arr_deriv = np.gradient(system_energy_arr, T_sol_arr)
joint_angles = -X_sol[:,:4]+X_sol[:,1:5]
joint_speeds = -X_sol[:,5:9]+X_sol[:,6:10]
external_power = np.sum(joint_speeds*U_sol[:,1:], axis=1)

plt.figure(figsize=[15,10])

plt.plot(T_sol_arr, system_energy_arr)
plt.plot(T_sol_arr, system_energy_arr_deriv)
plt.plot(T_sol_arr, external_power)
plt.grid()

x_guess = X_sol.copy()
x_guess[:,:5] = q_guess
x_guess[:,5:] = q_dot_guess

plt.figure(figsize=[15,15])
plt.grid()
for ii in range(N+1):
    points_x, points_y = chain_to_draw(x_guess[ii], params)
    plt.plot(points_x, points_y, color = plt.cm.viridis(ii/26))
    

plt.gca().set_aspect('equal')

In [ ]:
from optibot.schemes import interpolated_array, interpolated_array_derivative
from optibot.analysis import dynamic_error_implicit

n_graph = 1000
t_arr = np.linspace(0,2,n_graph)
dyn_err_q_hs_lin, dyn_err_v_hs_lin, dyn_err_2_a_hs_lin, dyn_err_2_b_hs_lin = dynamic_error_implicit(
    x_arr=X_sol,
    u_arr=U_sol,
    t_end=T_sol,
    params = params,
    F = F_impl_np,
    M = mass_matrix_np,
    lambda_arr = None,
    scheme = 'hs_parab', 
    u_scheme='parab',
    scheme_params={'u_c':U_c_sol,
                  'x_dot_c': X_dot_c_sol,
                  'x_c': X_c_sol},
    n_interp= n_graph)


for ii in range(5):
    plt.figure(figsize=[14,8])
    plt.plot(t_arr,-dyn_err_q_hs_lin[:,ii], '-', label = f'q {ii}')
    plt.plot(np.linspace(0,2,26), np.zeros(26), 'ok')
    plt.legend()
    plt.grid()


for ii in range(5):
    plt.figure(figsize=[14,8])
    plt.plot(t_arr,dyn_err_v_hs_lin[:,ii], '-', label = f'q {ii}')
    plt.plot(np.linspace(0,2,26), np.zeros(26), 'ok')
    plt.legend()
    plt.grid()
    plt.figure(figsize=[14,8])


for ii in range(5):
    plt.figure(figsize=[14,8])
    plt.plot(t_arr,dyn_err_2_b_hs_lin[:,ii], '-', label = f'q {ii}')
    plt.plot(np.linspace(0,2,26), np.zeros(26), 'ok')
    plt.legend()
    plt.grid()
    plt.figure(figsize=[14,8])


## Sistematic comparative

In [ ]:
from functools import lru_cache

In [ ]:
schemes = ['LG','LG2']#'D2', 'LGL', 
initials = ['lin']#, 'funcs']
solve_repetitions = 1
N_arr = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]#,21,22,23,24,25,30,35,40,45]#,50,55]#

results = {}

for scheme in schemes:
    key = scheme
    print('Problem:', key)
    results[key] = {'N_arr':N_arr}
    for N in N_arr:
        print(f'\tN = {N} , ', time.strftime('%H:%M:%S ', time.localtime(time.time())))
        results[key][N] = five_link(
            N = N,
            scheme = scheme,
            solve_repetitions = solve_repetitions,
            t_end = 0.7,
            step_length = 0.5)

In [ ]:
for scheme in schemes:
    key = scheme
    _c = []
    _cpudt = []
    for N in results[key]['N_arr']:
        _c.append(results[key][N]['cost'])
        _cpudt.append(results[key][N]['cpudt'])
    results[key]['cost'] = np.array(_c)
    results[key]['cpudt'] = np.array(_cpudt)

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams.update({'font.size': 15})
oct_fig_size = [10,6]

In [ ]:
plt.figure(figsize=oct_fig_size)
for scheme in schemes:
    key = scheme
    plt.plot(results[key]['N_arr'], results[key]['cost'][:], marker = 'o', label = scheme)
plt.yscale('log')
plt.grid()
plt.legend()

In [ ]:
#titles = ['LG','LG2']#'D2', 'LGL', 
plt.figure(figsize=oct_fig_size)
for ii in range(len(schemes)):
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'cpudt'], marker = 'o', c = f'C{ii}',label = schemes[ii])
plt.grid()
plt.legend()
#plt.yscale('log')
plt.title('Biped optimization time')
plt.xlabel('Number of collocation points')
plt.ylabel('Time (s)')
plt.tight_layout(pad = 0.0)
#plt.savefig(r'LG2pics/Biped_optimization_time_vs_interval_number.pdf', format='pdf')

In [ ]:
from optibot.pseudospectral import bary_poly, base_points, coll_points
def plot_u(uu, scheme, tf, color = 'k', t0 = 0, N_interp = 300):
    N = len(uu)
    tau = coll_points(N, scheme, 30)
    b_pol = bary_poly(tau, uu)
    coll_p_t = t0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(coll_p_t, uu, color+'o', label = 'control '+scheme)
    
    plt.plot(t_int_arr, b_pol(tau_int_arr), color)
    
def plot_x(qq, scheme, tf, color = 'k', t0 = 0, N_interp = 300, label = 'q'):
    N = len(qq)
    #tau = get_taus(N, scheme)
    tau = base_points(N, scheme, 30)
    b_pol = bary_poly(tau, qq)
    coll_p_t = t0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(coll_p_t, qq, color+'o', label = label + ' ' +scheme)
    
    plt.plot(t_int_arr, b_pol(tau_int_arr), color)

In [ ]:
for ii in range(5):

    N = 11
    plt.figure(figsize=[16,10])
    for jj in range(len(schemes)):
        scheme = schemes[jj]
        key = scheme
        plot_x(results[key][N]['q'][:,ii], scheme, 0.7, color = f'C{jj}', t0 = 0, N_interp = 300)
    plt.plot(_res['t'],_res['x'][:,ii], 'k:', label = 'HS reference')
        
    plt.grid()
    plt.legend()

In [ ]:
for ii in range(5):

    N = 11
    plt.figure(figsize=[16,10])
    for jj in range(len(schemes)):
        scheme = schemes[jj]
        key = scheme
        plot_u(results[key][N]['u'][:,ii], scheme, 0.7, color = f'C{jj}', t0 = 0, N_interp = 300)
    plt.plot(_res['t'],_res['u'][:,ii], 'k:', label = 'HS reference')
        
    plt.grid()
    plt.legend()

plt.figure(figsize=[15,15])
plt.grid()
N = 20
for ii in  range(0, N+1, 2):
    points_x, points_y = chain_to_draw(results['LG'][N]['q'][ii], params)
    plt.plot(points_x, points_y, marker = 'o', color = plt.cm.viridis(ii/N))
    

plt.gca().set_aspect('equal')

In [ ]:
from optibot.analysis import generate_G

def generate_G(M, F):
    """
    Generate a function G from M and F, so that from 

            | q''  |   |                | -1   |                 |
            |      | = |  M(x, params)  |    @ | F(x, u, params) |
            |lambda|   |                |      |                 |,
    
    we can get a function G so that:
        
         q'' = G(x, u) = (M(x)^-1 @  F(x, u)) [upperside]
            
    Parameters
    ----------
    M : Function of (x, params)
        Returns a Numerical Matrix.
    F : Function of (x, u, params)
        Returns a Numerical Vector.

    Returns
    -------
    G : Function of (x, u, params)
        Equal to q'' where the collocation constraint is enforced.

    """

    def G(x, u, params):
        dim = x.shape[-1] // 2
        mm = M(x, params)
        ff = F(x, u, params)
        res = np.linalg.solve(mm,ff)
        return res[:dim]
    return G

In [ ]:
G_nump = generate_G(F_impl = F_impl_np,
           M = mass_matrix_np,)

In [ ]:
def G_nump_q(q,v,u,params):
    q = np.array(q)
    v = np.array(v)
    x = np.concatenate((q,v), axis = -1)
    return G_nump(x,u,params)

In [ ]:
from optibot.pseudospectral import dynamic_error_pseudospectral, try_array_f,interpolations_pseudospectral

In [ ]:
_interpolations = interpolations_pseudospectral

from numpy import array, piecewise, linspace, expand_dims
from optibot.pseudospectral import extend_u_array, get_hermite_x, get_pol_u, get_pol_x
from optibot.pseudospectral import extend_x_arrays
from optibot.schemes import interp_2d
def _interpolations(
    qq,
    vv,
    uu,
    scheme,
    t0,
    t1,
    u_interp="pol",
    x_interp="pol",
    g_func=lambda q, v, u, p: u,
    params=[],
    n_interp=5000,
):
    """
    Generates arrays of equispaced points with values of dynamic error.
    
    If x(t) = [q(t), v(t)], and the physics equation states that x' = F(x, u),
    which is equivalent to [q', v'] = [v , G(q, v, u)] we can define the 
    dynamic errors at a point t as:
        dyn_q_err = q'(t) - v(t)
        dyn_v_err = v'(t) - G(q(t), v(t), u(t))
        dyn_2_err = q''(t) - G(q(t), v(t), u(t))
        
    'x_interp' and 'u_interp' define the way in which we interpolate the values
    of q, v and u between the given points.

    Parameters
    ----------
    qq : Numpy Array, shape = (W, N)
        Values known of q(t)
    vv : Numpy Array, shape = (W, N)
        Values known of v(t)
    uu : Numpy Array, shape = (Y, [Z])
        Values known of x(t)
    scheme : str
        Pseudospectral cheme used in the optimization.
        Acceptable values are:
            'LG'
            'LG_inv'
            'LGR'
            'LGR_inv'
            'LGL'
            'LGLm'
            'LG2'
            'D2'
    t0 : float
        starting time of interval of analysis
    t1 : float
        ending time of interval of analysis
    u_interp :  string, optional
        Model of the interpolation that must be used. The default is "pol".
        Acceptable values are:
            "pol": corresponding polynomial interpolation
            "lin": lineal interpolation
            "smooth": 3d order spline interpolation
    x_interp : string, optional
        Model of the interpolation that must be used. The default is "pol".
        Acceptable values are:
            "pol": corresponding polynomial interpolation
            "lin": lineal interpolation
            "Hermite": Hermite's 3d order spline interpolation
    g_func : Function of (q, v, u, params)
        A function of a dynamic sistem, so that
            q'' = g(q, q', u, params)
    params : list
        Physical problem parameters to be passed to F
    n_interp : int, default 5000
        number of interpolation points

    Raises
    ------
    NameError
        When an unsupported value for scheme, x_interp or u_interp is used.

    Returns
    -------
    err_q : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q'(t) - v(t).
    err_v : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error v'(t) - G(q(t), v(t), u(t)).
    err_2 : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q''(t) - G(q(t), q'(t), u(t)).

    """
    from scipy.interpolate import CubicHermiteSpline as hermite
    from numpy import interp, gradient, zeros_like

    scheme_opts = ["LG", "LG_inv", "LGR", "LGR_inv", "LGL", "D2", "LG2", "LGLm"]
    if scheme not in scheme_opts:
        NameError(f"Invalid scheme.\n valid options are {scheme_opts}")

    N = len(qq)
    tau_arr = linspace(-1, 1, n_interp)

    g_func = try_array_f(g_func)

    if u_interp == "pol":
        pol_u = get_pol_u(scheme, uu)
        u_arr = pol_u(tau_arr)
    elif u_interp == "lin":
        tau_u, uu = extend_u_array(uu, scheme, N)
        if len(uu.shape) == 1:
            u_arr = interp(tau_arr, tau_u, uu)
        elif len(uu.shape) == 2:
            u_arr = interp_2d(tau_arr, tau_u, uu)
        else:
            raise ValueError(
                f"U has {len(uu.shape)} dimensions, values accepted are 1 and 2"
            )
    elif u_interp == "smooth":
        tau_u, uu = extend_u_array(uu, scheme, N)
        uu_dot = gradient(uu, tau_u)
        u_arr = hermite(tau_u, uu, uu_dot)(tau_arr)
    else:
        raise NameError(
            'Invalid interpolation method for u.\n valid options are "pol", "lin", "smooth"'
        )

    if x_interp == "pol":
        tau_x = base_points(N, scheme)
        pol_q, pol_v, pol_q_d, pol_v_d, pol_q_d_d = get_pol_x(scheme, qq, vv, t0, t1)
        q_arr = pol_q(tau_arr)
        v_arr = pol_v(tau_arr)
        q_arr_d = pol_q_d(tau_arr)
        v_arr_d = pol_v_d(tau_arr)
        q_arr_d_d = pol_q_d_d(tau_arr)
    elif x_interp == "lin":
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        if len(qq.shape) == 1:
            q_arr = interp(tau_arr, tau_x, qq)
            v_arr = interp(tau_arr, tau_x, vv)
        elif len(qq.shape) == 2:
            q_arr = interp_2d(tau_arr, tau_x, qq)
            v_arr = interp_2d(tau_arr, tau_x, vv)
        else:
            raise ValueError(
                f"q has {len(qq.shape)} dimensions, values accepted are 1 and 2"
            )
        
        coll_p = t0 + (1 + array(tau_x, dtype="float64")) * (t1 - t0) / 2
        t_arr_lin = linspace(t0, t1, n_interp)
        q_arr_d = find_der_polyline(t_arr_lin, coll_p, qq)
        v_arr_d = find_der_polyline(t_arr_lin, coll_p, vv)
        q_arr_d_d = zeros_like(q_arr)
    elif x_interp == "Hermite":
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        aa = g_func(qq, vv, uu, params)
        her_q, her_v, her_q_d, her_v_d, her_q_d_d = get_hermite_x(
            qq, vv, aa, tau_x, t0, t1
        )
        t_arr_lin = linspace(t0, t1, n_interp)
        q_arr = her_q(t_arr_lin)
        v_arr = her_v(t_arr_lin)
        q_arr_d = her_q_d(t_arr_lin)
        v_arr_d = her_v_d(t_arr_lin)
        q_arr_d_d = her_q_d_d(t_arr_lin)
    else:
        raise NameError(
            'Invalid interpolation method for x.\n valid options are "pol", "lin", "Hermite"'
        )
    
    return q_arr, q_arr_d, v_arr, v_arr_d, q_arr_d_d, u_arr

In [ ]:
_q_arr, _q_arr_d, _v_arr, _v_arr_d, _q_arr_d_d, _u_arr = _interpolations(
    results[key][N]['q'],
    results[key][N]['v'],
    results[key][N]['u'],
    scheme,
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = 1000,
)

In [ ]:
for ii in range(5):
#ii = 1

    plt.figure(figsize=[16,10])
    plot_x(results[key][N]['q'][:,ii],scheme, 2, color = 'k', t0 = 0, N_interp = 300)
    plt.plot(np.linspace(0, 2, 1000), _q_arr[:,ii], label = 'q_arr')
    plt.grid()
    plt.legend()

for ii in range(5):
#ii = 1

    plt.figure(figsize=[16,10])
    plot_x(results[key][N]['v'][:,ii],scheme, 2, color = 'k', t0 = 0, N_interp = 300)
    plt.plot(np.linspace(0, 2, 1000), _v_arr[:,ii], label = 'v_arr')
    plt.plot(np.linspace(0, 2, 1000), _q_arr_d[:,ii], label = 'q_d_arr')
    plt.grid()
    plt.legend()

In [ ]:
from optibot.pseudospectral import try_array_f
def _g_func(q,v,u,params):
    return -LM_small.dynam_g_q(q,v,0,u,[],params)
_g_func = try_array_f(_g_func)

In [ ]:
_g_func = try_array_f(G_nump_q)

In [ ]:
for ii in range(5):

    plt.figure(figsize=[16,10])
    plt.plot(np.linspace(0, 2, 1000), _g_func(_q_arr, _v_arr, _u_arr, params)[:,ii], label = 'v_arr')
    plt.plot(np.linspace(0, 2, 1000), _q_arr_d_d[:,ii], label = 'q_d_d_arr')
    plt.grid()
    plt.legend()

In [ ]:
G_nump_q

In [ ]:
_d_e = dynamic_error_pseudospectral(
    results[key][N]['q'],
    results[key][N]['v'],
    results[key][N]['u'],
    scheme,
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = 1000,
)

In [ ]:
for ii in range(3):
    plt.figure(figsize=[14,10])
    plt.plot(np.linspace(-1, 1, 1000), _d_e[ii])
    plt.grid()
    plt.plot(coll_points(N, scheme), np.zeros_like(coll_points(N, scheme)), 'ok')

## Trapz abs err + integr

In [ ]:
N= 10
n_interp = 1000
_d_e = dynamic_error_pseudospectral(
    results['LG'][N]['q'],
    results['LG'][N]['v'],
    results['LG'][N]['u'],
    'LG',
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = n_interp,
)
_d_e_2 = dynamic_error_pseudospectral(
    results['LG2'][N]['q'],
    results['LG2'][N]['v'],
    results['LG2'][N]['u'],
    'LG2',
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = n_interp,
)


titles = [
    r'First order dynamic error $\varepsilon^{[1]}$,'+f' N = {N}',
    r'$\dot{v} - g(q, v, u),$'+f' N = {N}',
    r'Second order dynamic error $\varepsilon^{[2]}$,'+f' N = {N}'
]
filenames = [
    f'first_order_dynamic_error_N_{N}',
    f'v_dot_g_diff_N_{N}',
    f'second_order_dynamic_error_N_{N}'
]
units = ['rad/s', 'rad/s^2', 'rad/s^2']


for ii in range(3):
    plt.figure(figsize=oct_fig_size)
    plt.plot(np.linspace(-1, 1, n_interp), np.sum(np.abs(_d_e[ii]), axis=1), c='C0', label = 'LG')
    plt.plot(np.linspace(-1, 1, n_interp), np.sum(np.abs(_d_e_2[ii]), axis=1),c = 'C1', label = 'LG2')
    plt.grid()
    plt.plot(
        coll_points(N, scheme),
        np.zeros_like(coll_points(N, scheme)),
        'ok')
    plt.legend()
    plt.title(titles[ii])
    plt.xlabel('Time(s)')
    plt.ylabel(f'Dynamic error $({units[ii]})$')
    plt.tight_layout(pad = 0.0)
#    plt.savefig(r'LG2pics/biped_'+filenames[ii]+f'.pdf', format='pdf')

In [ ]:
n_interp = 2000
for scheme in schemes:
    key = scheme 
    print('scheme: ', scheme)
    for N in results[key]['N_arr']:
        print(f'\t N:{N} , ', time.strftime('%H:%M:%S ', time.localtime(time.time())))
        _dyn_err = dynamic_error_pseudospectral(
            results[key][N]['q'],
            results[key][N]['v'],
            results[key][N]['u'],
            scheme,
            t0 = 0,
            t1 = 0.7,
            u_interp="pol",
            x_interp="pol",
            g_func=G_nump_q,
            params=params,
            n_interp = n_interp,
        )
        results[key][N]['err_q_arr'] = _dyn_err[0]
        results[key][N]['err_v_arr'] = _dyn_err[1]
        results[key][N]['err_2_arr'] = _dyn_err[2]

In [ ]:
for scheme in schemes:
    key = scheme
    print('scheme: ', scheme)
    _lis = []
    for N in results[key]['N_arr']:
        #print('\t N:', N)
        integ_err = np.trapz(np.abs(results[key][N]['err_2_arr']), np.linspace(0,2,n_interp), axis = 0)
        results[key][N]['err_2_abs_integ'] = integ_err
        _lis.append(integ_err)
    results[key]['err_2_abs_integ_vect'] = np.array(_lis)
    results[key]['err_2_abs_integ'] = np.sum(np.array(_lis), axis = 1)

In [ ]:
for ii in range(5):
    plt.figure(figsize=[16,10])
    for scheme in schemes:
        key = scheme
        plt.plot(results[key]['N_arr'], results[key]['err_2_abs_integ_vect'][:,ii], marker = 'o', label = scheme)
    plt.yscale('log')
    plt.grid()
    plt.legend()

In [ ]:
plt.figure(figsize=oct_fig_size)
for scheme in schemes:
    key = scheme
    plt.plot(results[key]['N_arr'], results[key]['err_2_abs_integ'], marker = 'o', label = scheme)
plt.yscale('log')
plt.grid()
plt.legend()
plt.title('Biped second order dynamic error $E^{[2]}$')
plt.xlabel('Number of collocation points')
plt.ylabel(f'Dynamic error ($rad/s$)')
plt.tight_layout(pad = 0.0)
#plt.savefig(r'LG2pics/biped_second_order_dynamic_error_vs_interval_number.pdf', format='pdf')

from optibot.schemes import _newpoint, _calculate_missing_arrays, interp_2d, _newpoint_der
from numpy import interp
def dynamic_error_point(
    x_arr,
    u_arr,
    t_arr,
    t,
    params,
    F,
    x_dot_arr=None,
    scheme="hs_scipy",
    u_scheme="lin",
    scheme_params={},
):
    """
    Generate arrays of equispaced points with values of dynamic error.
    
    If x(t) = [q(t), v(t)], and the physics equation states that x' = F(x, u),
    which is equivalent to [q', v'] = [v , G(q, v, u)] we can define the 
    dynamic errors at a point t as:
        dyn_q_err = q'(t) - v(t)
        dyn_v_err = v'(t) - G(q(t), v(t), u(t))
        dyn_2_err_a = q''(t) - G(q(t), v(t), u(t))
        dyn_2_err_b = q''(t) - G(q(t), q'(t), u(t))
        
    'scheme' and 'u_scheme' define the way in which we interpolate the values
    of q, v and u between the given points.
        
    It is assumed that X and U start at t = 0 and are equispaced in time
    in the interval (0, t_end).
    

    Parameters
    ----------
    x_arr : Numpy Array, shape = (W, 2N)
        Values known of x(t)
    u_arr : Numpy Array, shape = (W, [Y])
        Values known of u(t)
    t_end : float
        ending time of interval of analysis
    params : list
        Physical problem parameters to be passed to F
    F : Function of (x, u, params)
        A function of a dynamic sistem, so that
            x' = F(x, u, params)
        if x_dot_arr is None and F is not, F will be used to calculate X'
    x_dot_arr : Numpy Array, optional, shape = (W, 2N), default = None
        Known values of X'
        if x_dot_arr is None, F will be used to calculate X'
    scheme : str, optional
        Scheme to be used in the X interpolation. The default is "hs_scipy".
        Acceptable values are:
            "trapz" : trapezoidal scheme compatible interpolation (not lineal!)
            "trapz_mod": modified trapezoidal scheme compatible interpolation (not lineal!)
            "hs_scipy": 3d order polynomial that satisfies continuity in x(t) and x'(t)
            "hs": Hermite-Simpson scheme compatible interpolation
            "hs_mod": modified Hermite-Simpson scheme compatible interpolation
            "hs_parab": Hermite-Simpson scheme compatible interpolation with parabolic U
            "hs_mod_parab": modified Hermite-Simpson scheme compatible interpolation with parabolic U
    u_scheme : string, optional
        Model of the interpolation that must be used. The default is "lin".
        Acceptable values are:
            "lin": lineal interpolation
            "parab": parabolic interpolation, requires central points array
            as scheme params[0]
    scheme_params :dict, optional
        Aditional parameters of the scheme. The default is {}.
    n_interp : int, optional
        Number of interpolation points. The default is 2000.

    Returns
    -------
    dyn_err_q : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q'(t) - v(t).
    dyn_err_v : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error v'(t) - G(q(t), v(t), u(t)).
    dyn_err_2_a : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q''(t) - G(q(t), v(t), u(t)).
    dyn_err_2_b : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q''(t) - G(q(t), q'(t), u(t)).

    """
    if "parab" in scheme and u_scheme == "lin":
        warnings.warn(
            "You are currently using a u-parabolic interpolation for x with a lineal interpolation of u"
        )
    if "parab" in u_scheme and "parab" not in scheme:
        warnings.warn(
            "You are currently using a parabolic interpolation for u with a non u-parabolic interpolation of x"
        )
    N = x_arr.shape[0] - 1
    dim = x_arr.shape[1] // 2
    h = (t_arr[-1]-t_arr[0]) / N
    
    x_dot_arr = _calculate_missing_arrays(
        X, U, h, params, F, x_dot_arr, scheme, u_scheme, scheme_params
    )
    if u_scheme in ["min_err", "pinv_dyn"]:
        scheme_params["X"] = X
        scheme_params["scheme"] = scheme
        scheme_params["params"] = params
        scheme_params["x_dot_arr"] = x_dot_arr
        if u_scheme == "min_err":
            if F is None:
                raise ValueError(
                    "F cannot be None when using min_err as u interpolation"
                )
            scheme_params["F"] = F
            
    
    if scheme == "hs_scipy":
        X_interp = hermite(t_array, x_arr, x_dot_arr)
        x = X_interp(t)
    else:
        x = array(
                _newpoint(x_arr, x_dot_arr, h, t, params, scheme, scheme_params)
            ).flatten()
        
    if u_scheme == "lin":
        if len(u_arr.shape) == 1:
            u = interp(t, t_array, u_arr)
        elif len(u_arr.shape) == 2:
            u = interp_2d(t, t_array, u_arr)
        else:
            raise ValueError(
                f"U has {len(u_arr.shape)} dimensions, values accepted are 1 and 2"
            )
    else:
        u = array(
            _newpoint_u(U, h, t, u_scheme, scheme_params)
        ).flatten()
        
        
    if scheme == "hs_scipy":
        X_interp = hermite(t_array, x_arr, x_dot_arr)
        X_dot_interp = X_interp.derivative()
        x_d = X_dot_interp(t)
        x_d_d = X_dot_interp.derivative()(t)
    else:
        x_d = array(
                _newpoint_der(x_arr, x_dot_arr, h, t, params, scheme, 1, scheme_params)
            ).flatten()
        x_d_d = array(
                _newpoint_der(x_arr, x_dot_arr, h, t, params, scheme, 2, scheme_params)
            ).flatten()
            
    
    f_a = F(x, u, params)[dim:]
    x_q = x.copy()
    x_q[dim:] = x_d[:dim]
    f_b = F(x_q, u, params)[dim:]
    dyn_err_q = x_d[:dim] - x[dim:]
    dyn_err_v = x_d[dim:] - f_a
    dyn_err_2_a = x_d_d[:dim] - f_a
    dyn_err_2_b = x_d_d[:dim] - f_b
    return dyn_err_q, dyn_err_v, dyn_err_2_a, dyn_err_2_b

In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 


n_graph = 3000
t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    if 'parab' in scheme:
        u_scheme = 'parab'
    else:
        u_scheme = 'lin'
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    for N in N_arr:
        print(f'\tN = {N}')
        dyn_err_q, dyn_err_v, _, dyn_err_2 = dynamic_error_implicit(
            x_arr=results[key][N]['x'],
            u_arr=results[key][N]['u'],
            t_end=results[key][N]['t'],
            params = params,
            F = F_impl_np,
            M = mass_matrix_np,
            scheme = scheme, 
            u_scheme = u_scheme,
            scheme_params={'u_c':results[key][N]['u_c'],
                          'x_dot_c': results[key][N]['x_dot_c'],
                          'x_c': results[key][N]['x_c']},
            n_interp= n_graph)
        results[key][N]['dyn_err_q'] = dyn_err_q
        results[key][N]['dyn_err_v'] = dyn_err_v
        results[key][N]['dyn_err_2'] = dyn_err_2

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams.update({'font.size': 15})
oct_fig_size = [10,6]

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

schemes = ['hs_parab','hs_mod_parab', 'trapz', 'trapz_mod']
titles = ['Hermite Simpson','2nd order Hermite Simpson', 'Trapezoidal', '2nd order Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for kk in range(len(schemes)):
    scheme = schemes[kk]
    plt.figure(figsize=[14,8])
    for ii in range(5):
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            plt.plot(t_arr[cut_p:jj],results[scheme][N]['dyn_err_q'][cut_p:jj,ii], '-', c = colors[ii], label = f'$q_{ii+1}$' if cut_p == 0 else None)
            cut_p = jj
    plt.plot(np.linspace(0,results[scheme][N]['t'],N+1), np.zeros(N+1), 'ok')
    plt.legend()
    plt.grid()
    if kk == 1:
        plt.ylim([-0.00001, 0.00001])
    elif kk == 3:
        plt.ylim([-0.001, 0.001])
    plt.title(r'First order dynamic error $\varepsilon^{[1]}_{q_i}$,'+f' {titles[kk]} scheme')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s)$')
    plt.tight_layout(pad = 0.0)
    sch_type = titles[kk].replace(' ','_')
    plt.savefig(f'5_link_First_Order_Dynamic_Error_{sch_type}_scheme.eps', format='eps')


schemes = ['hs_parab','hs_mod_parab', 'trapz', 'trapz_mod']
titles = ['Hermite Simpson','2nd order Hermite Simpson', 'Trapezoidal', '2nd order Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for kk in range(len(schemes)):
    scheme = schemes[kk]
    plt.figure(figsize=[14,8])
    for ii in range(5):
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            plt.plot(t_arr[cut_p:jj],results[scheme][N]['dyn_err_2'][cut_p:jj,ii], '-', c = colors[ii], label = f'$q_{ii+1}$' if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    if 'hs' in scheme:
        plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='b', label = 'collocation points')
        plt.ylim([-0.08, 0.08])
    else:
        plt.ylim([-1.75, 1.75])
    plt.legend()
    plt.grid()
    plt.title(r'Second order dynamic error $\varepsilon^{{[2]}}_{{q_i}}$,'+f' {titles[kk]} scheme')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s^2)$')
    plt.tight_layout(pad = 0.0)
    sch_type = titles[kk].replace(' ','_')
    plt.savefig(f'5_link_Second_Order_Dynamic_Error_{sch_type}_scheme.eps', format='eps')


In [ ]:
def arr_mod(x):
    x_1 = np.sum(x*x, axis=1)
    return np.sqrt(x_1)
def arr_sum(x):
    return np.sum(np.abs(x), axis = 1)
def arr_max(x):
    return np.max(np.abs(x), axis = 1)

In [ ]:
schemes = ['hs_mod_parab','hs_parab']#, 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson','Hermite Simpson']#, 'Trapezoidal', 'Modified Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['dyn_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Second order dynamic error $\varepsilon^{[2]}$,'+f' N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s^2)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_HS_N{N}_second_order_dynamic_error.pdf', format='pdf')
    


In [ ]:
schemes = ['trapz', 'trapz_mod']#, 'trapz', 'trapz_mod']
titles = ['Trapezoidal', '2nd order Trapezoidal']#, 'Trapezoidal', 'Modified Trapezoidal']
funcs = [arr_sum,]#arr_mod,  arr_max
#func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 50
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in range(2):
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['dyn_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk+2}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    #plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Second order dynamic error $\varepsilon^{[2]}$,'+f' N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s^2)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_Trapezoidal_N{N}_second_order_dynamic_error.pdf', format='pdf')
    


In [ ]:
def state_error(t_arr, dyn_err, N,):
    n_int = len(t_arr)
    interv_n = (N * t_arr)//2
    err = []
    cut_p = 0
    for ii in range(1,N+1):
        jj = np.searchsorted(interv_n, ii)
        err_point = np.trapz(np.abs(dyn_err[cut_p:jj+1]), t_arr[cut_p:jj+1], axis=0)
        err.append(err_point)
        cut_p = jj
    err = np.array(err, dtype = float)
    return err

def total_state_error(t_arr, dyn_err):
    errors = np.trapz(dyn_err, t_arr, axis=0)
    return errors


def total_state_error_abs(t_arr, dyn_err):
    errors = np.trapz(np.abs(dyn_err), t_arr, axis=0)
    return errors

In [ ]:
def integ_state_error(t_arr, dyn_err):
    int_x = np.zeros_like(dyn_err)
    arr_len = dyn_err.shape[0]
    for ii in range(1,arr_len):
        int_x[ii,:] = np.trapz(dyn_err[:ii+1,:], t_arr[:ii+1], axis=0)
    return int_x

In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 

t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    for N in N_arr:
        print(f'\tN = {N}')
        for letter in 'qv2':
            results[key][N][f'integ_dyn_err_{letter}']= total_state_error(t_arr, results[scheme][N][f'dyn_err_{letter}'])
            results[key][N][f'module_dyn_err_{letter}']= np.sqrt(np.dot(results[key][N][f'integ_dyn_err_{letter}'], results[key][N][f'integ_dyn_err_{letter}']))
            results[key][N][f'sum_dyn_err_{letter}'] = np.sum(np.abs(results[key][N][f'integ_dyn_err_{letter}']))
        

In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 

t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    for N in N_arr:
        print(f'\tN = {N}')
        results[key][N]['integ_err_2'] = integ_state_error(t_arr, results[key][N]['dyn_err_2'])
        results[key][N]['twice_integ_err_2'] = integ_state_error(t_arr, results[key][N]['integ_err_2'])
        

In [ ]:
sch = [['hs_parab','hs_mod_parab'],['trapz', 'trapz_mod']]
tit = [['Hermite Simpson','2nd order Hermite Simpson'],['Trapezoidal', '2nd order Trapezoidal']]
colors = [f'C{ii}' for ii in [1,0,2,3]]
n_int = len(t_arr)
N_hh = [25,50]
t_end = 0.7
for hh in range(2):
    schemes = sch[hh]
    titles = tit[hh]
    N = N_hh[hh]
    interv_n = (N * t_arr)/2
    for ii in range(5):
        plt.figure(figsize=oct_fig_size)
        for kk in range(len(schemes)):
            scheme = schemes[kk]
            key = scheme
            cut_p = 0
            for ll in range(1,N+1):
                jj = np.searchsorted(interv_n, ll)
                plt.plot(t_arr[cut_p:jj],results[key][N]['integ_err_2'][cut_p:jj,ii], '-', c = colors[2*hh+kk], label = titles[kk] if cut_p == 0 else None)
                cut_p = jj
        plt.plot(np.linspace(0,t_end,N+1), np.zeros(N+1), 'ok', label = 'knot & collocation points')
        if hh == 0:
            plt.plot(np.linspace(0,t_end,2*N+1)[1::2], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
        plt.legend()
        plt.grid()
        #plt.ylim([-0.00022, 0.00022])
        plt.title(r'Integrated Second order dynamic error $\int_{0}^{t}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau$,'+f'\n{titles[0]} schemes, N = {N}')
        plt.xlabel('Time(s)')
        units = 'm/s' if ii == 0 else'rad/s'
        plt.ylabel(f'Dynamic error $({units})$')
        plt.tight_layout(pad = 0.0)
        sch_type = titles[0].replace(' ','_')
        plt.savefig(f'Biped_Integrated_Second_Order_Dynamic_Error_q_{ii+1}_{sch_type}_schemes_N_{N}.png', format='png')

In [ ]:
sch = [['hs_parab','hs_mod_parab'],['trapz', 'trapz_mod']]
tit = [['Hermite Simpson','2nd order Hermite Simpson'],['Trapezoidal', '2nd order Trapezoidal']]
colors = [f'C{ii}' for ii in [1,0,2,3]]
n_int = len(t_arr)
N_hh = [25,50]
t_end = 0.7
for hh in range(2):
    schemes = sch[hh]
    titles = tit[hh]
    N = N_hh[hh]
    interv_n = (N * t_arr)/2
    for ii in range(5):
        plt.figure(figsize=oct_fig_size)
        for kk in range(len(schemes)):
            scheme = schemes[kk]
            key = scheme
            cut_p = 0
            for ll in range(1,N+1):
                jj = np.searchsorted(interv_n, ll)
                plt.plot(t_arr[cut_p:jj],results[key][N]['twice_integ_err_2'][cut_p:jj,ii], '-', c = colors[2*hh+kk], label = titles[kk] if cut_p == 0 else None)
                cut_p = jj
        plt.plot(np.linspace(0,t_end,N+1), np.zeros(N+1), 'ok', label = 'knot & collocation points')
        if hh == 0:
            plt.plot(np.linspace(0,t_end,2*N+1)[1::2], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
        plt.legend()
        plt.grid()
        #plt.ylim([-0.00022, 0.00022])
        plt.title(r'Twice integrated second order dynamic error $\int_{0}^{t}\int_{0}^{\xi}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau d\xi$,'+f'\n{titles[0]} schemes, N = {N}')
        plt.xlabel('Time(s)')
        units = 'm' if ii == 0 else'rad'
        plt.ylabel(f'Dynamic error $({units})$')
        plt.tight_layout(pad = 0.0)
        sch_type = titles[0].replace(' ','_')
        plt.savefig(f'Biped_Twice_Integrated_Second_Order_Dynamic_Error_q_{ii+1}_{sch_type}_schemes_N_{N}.png', format='png')

In [ ]:
schemes = ['hs_mod_parab','hs_parab']#, 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson','Hermite Simpson']#, 'Trapezoidal', 'Modified Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute Integrated Second order dynamic error $\Sigma |\int_{0}^{t}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau|$,'+f' HS schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_HS_N{N}_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['trapz', 'trapz_mod']#
titles = ['Trapezoidal', 'Modified Trapezoidal']#
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk+2}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute Integrated Second order dynamic error $\Sigma |\int_{0}^{t}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau|$,'+f' Trapz schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_Trapz_N{N}_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['hs_mod_parab','hs_parab']#, 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson','Hermite Simpson']#, 'Trapezoidal', 'Modified Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['twice_integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute twice integrated second order dynamic error $\Sigma |\int_{0}^{t}\int_{0}^{\xi}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau d\xi|$,'+f' HS schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_HS_N{N}_twice_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['trapz', 'trapz_mod']#
titles = ['Trapezoidal', 'Modified Trapezoidal']#
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['twice_integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk+2}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute twice integrated second order dynamic error $\Sigma |\int_{0}^{t}\int_{0}^{\xi}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau d\xi|$,'+f' Trapz schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_Trapz_N{N}_twice_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 

t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    _sum = []
    for N in N_arr:
        results[key][N]['twice_integ_dyn_err_2'] = results[key][N]['twice_integ_err_2'][-1,:]
        results[key][N][f'sum_twice_dyn_err_2'] = np.sum(np.abs(results[key][N]['twice_integ_dyn_err_2']))
        _sum.append(results[key][N][f'sum_twice_dyn_err_2'])
    results[key][f'sum_twice_dyn_err_2_array'] = np.array(_sum)

In [ ]:
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    
    for letter in 'qv2':
        list_mod = []
        list_sum = []
        for N in N_arr:
            #print(f'\tN = {N}')
            list_mod.append(results[key][N][f'module_dyn_err_{letter}'])
            list_sum.append(results[key][N][f'sum_dyn_err_{letter}'])
        results[key][f'module_dyn_err_{letter}_array'] = np.array(list_mod)
        results[key][f'sum_dyn_err_{letter}_array'] = np.array(list_sum)

In [ ]:
for scheme in results.keys():
    if 'hs' in scheme:
        n_coll = np.array(results[scheme]['N_arr'])*2-1
        results[scheme]['N_coll_arr'] = n_coll
    else:
        results[scheme]['N_coll_arr'] = results[scheme]['N_arr']
    print(scheme, results[scheme]['N_arr'],results[scheme]['N_coll_arr'])

schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=[14,8])
for ii in range(4):
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'module_dyn_err_2_array'], marker = 'o',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('Module second order dynamic error')
plt.xlabel('Number of intervals')
plt.ylabel('Dynamic error ($rad/s^2$)')
plt.savefig(f'5_link_Module_second_order_dynamic_error_vs_interval_number.eps', format='eps')

In [ ]:
schemes_graph = ['hs_mod_parab', 'hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
colors = [f'C{ii}' for ii in range(9)]
data_array = ['err_q_acum','err_v_acum','err_2_b_acum','cpudt']
initial = 'lin'


data_key = data_array[2]
for qq in range(5):
    #plt.figure(figsize=[10,6])
    #plt.title(f'Biped second order dynamic error $|E^{{*[2]}}_{{q_{qq+1}}}|$')
    for ii in [2,3,1,0]:
        scheme = schemes_graph[ii]
        key = scheme #+ '_' + initial
        print(titles[ii], f'Error 2 q_{qq+1}:')
        for nn in results[key]['N_arr']:
            print('\tN:', nn, 'Error:', results[key][nn]['integ_dyn_err_2'][qq])
        N_arr = results[key]['N_arr']
    #    if len(results[key][data_key].shape) == 1:
    #        plt.plot(N_arr,np.abs(results[key][data_key]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #    else:
    #        plt.plot(N_arr,np.abs(results[key][data_key][:,qq]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #plt.yscale('log')
    #plt.xlabel('Number of intervals')
    #plt.grid()
    #plt.legend()
    units = 'm/s' if qq == 0 else'rad/s'
    #plt.ylabel(f'Dynamic error $({units})$')
    #plt.tight_layout(pad = 0.0)
    #plt.savefig(f'Cartpole_Integrated_Second_Order_Dynamic_Error_q_{qq+1}_vs_N.pdf', format='pdf')


In [ ]:
schemes_graph = ['hs_mod_parab', 'hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
colors = [f'C{ii}' for ii in range(9)]
data_array = ['err_q_acum','err_v_acum','err_2_b_acum','cpudt']
initial = 'lin'


data_key = data_array[2]
for qq in range(5):
    #plt.figure(figsize=[10,6])
    #plt.title(f'Biped second order dynamic error $|E^{{*[2]}}_{{q_{qq+1}}}|$')
    for ii in [2,3,1,0]:
        scheme = schemes_graph[ii]
        key = scheme #+ '_' + initial
        print(titles[ii], f'Error 2 q_{qq+1}:')
        for nn in results[key]['N_arr']:
            print('\tN:', nn, 'Error:', results[key][nn]['twice_integ_dyn_err_2'][qq])
        N_arr = results[key]['N_arr']
    #    if len(results[key][data_key].shape) == 1:
    #        plt.plot(N_arr,np.abs(results[key][data_key]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #    else:
    #        plt.plot(N_arr,np.abs(results[key][data_key][:,qq]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #plt.yscale('log')
    #plt.xlabel('Number of intervals')
    #plt.grid()
    #plt.legend()
    units = 'm/s' if qq == 0 else'rad/s'
    #plt.ylabel(f'Dynamic error $({units})$')
    #plt.tight_layout(pad = 0.0)
    #plt.savefig(f'Cartpole_Integrated_Second_Order_Dynamic_Error_q_{qq+1}_vs_N.pdf', format='pdf')


In [ ]:
schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=oct_fig_size)
for ii in [2,3,1,0]:
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'sum_dyn_err_2_array'], marker = 'o', c = f'C{ii}',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('5-link biped second order dynamic error $\Sigma |E^{*[2]}_i|$')
plt.xlabel('Number of intervals')
plt.ylabel('Dynamic error ($rad/s$)')
plt.tight_layout(pad = 0.0)
plt.savefig(f'5_link_Sum_second_order_dynamic_error_vs_interval_number.png', format='png')

In [ ]:
schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=oct_fig_size)
for ii in [2,3,1,0]:
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'sum_twice_dyn_err_2_array'], marker = 'o', c = f'C{ii}',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('5-link biped twice integrated second order dynamic error $\Sigma |EE^{*[2]}_i|$')
plt.xlabel('Number of intervals')
plt.ylabel('Dynamic error ($rad$)')
plt.tight_layout(pad = 0.0)
plt.savefig(f'5_link_Sum_twice_integ_second_order_dynamic_error_vs_interval_number.png', format='png')

schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['Modified Hermite Simpson', 'Hermite Simpson','Trapezoidal', 'Modified Trapezoidal']
plt.figure(figsize=[14,8])
for ii in range(4):
    key = schemes[ii]
    plt.plot(results[key]['N_coll_arr'], results[key][f'sum_dyn_err_2_array'], marker = 'o',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('Added second order dynamic error')
plt.xlabel('Number of collocation points')
plt.ylabel('Dynamic error ($rad/s^2$)')
plt.savefig(f'5_link_Sum_second_order_dynamic_error_vs_collocation_points_number.eps', format='eps')

In [ ]:
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    list_cpudt = []
    for N in N_arr:
        #print(f'\tN = {N}')
        list_cpudt.append(results[key][N]['cpudt'])
    results[key]['cpudt_array'] = np.array(list_cpudt)

In [ ]:
schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=oct_fig_size)
for ii in [2,3,1,0]:
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'cpudt_array'], marker = 'o', c = f'C{ii}', label = titles[ii])
plt.grid()
plt.legend()
#plt.yscale('log')
plt.title('5-link biped optimization time')
plt.xlabel('Number of intervals')
plt.ylabel('Time (s)')
plt.tight_layout(pad = 0.0)
plt.savefig(f'5_link_optimization_vs_interval_number.pdf', format='pdf')

schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['Modified Hermite Simpson', 'Hermite Simpson','Trapezoidal', 'Modified Trapezoidal']
plt.figure(figsize=[14,8])
for ii in range(4):
    key = schemes[ii]
    plt.plot(results[key]['N_coll_arr'], results[key][f'cpudt_array'], marker = 'o',label = titles[ii])
plt.grid()
plt.legend()
#plt.yscale('log')
plt.title('optimization time')
plt.xlabel('Number of collocation points')
plt.ylabel('time (s)')
plt.savefig(f'5 link optimization vs collocation points number.eps', format='eps')

In [ ]:
for scheme in ['hs_mod_parab', 'hs_parab', 'trapz', 'trapz_mod']:
    key = scheme
    for N in [25,50]:#results[key]['N_arr']:
        print('scheme:', scheme, 'N:', N,'\n\ttime:', results[key][N][f'cpudt'],
              '\n\tErr 1:', results[key][N]['sum_dyn_err_q'], '\n\tErr 2:', results[key][N]['sum_dyn_err_2'])

## Animation

In [ ]:
from matplotlib import animation, rc
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from IPython.display import HTML

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
def body_tray(X, params):
    res = []
    for ii in range(X.shape[0]):
        res.append(list(chain_to_draw(X[ii,:], params)))
    return np.array(res)

In [ ]:
def loop_body_tray(X, params):
    point_tray = body_tray(X, params)
    point_tray_loop = np.append(
        point_tray,
        np.expand_dims(
            np.array(list(chain_to_draw(X[0,[4,3,2,1,0,5,6,7,8,9]],params)))
            ,0),
        0)
    return point_tray_loop

In [ ]:
def create_anim(X, U, params, n_loops = 1):
    [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
    ] = params
    
    N = X.shape[0]
    fig, ax = plt.subplots()
    draw_width = 14
    draw_height = 14
    
    fig.set_dpi(300)
    fig.set_size_inches([draw_width,draw_height])
    ax.set_xlim(( -1, 1))
    ax.set_ylim(( -0.2, 1.8))
    
    body, = ax.plot([], [], lw=4, ms = 12, marker = 'o')
    trail, = ax.plot([], [], lw=1, color = 'k')
    old_trail, = ax.plot([], [], lw=1, color = 'k')
    next_trail, = ax.plot([], [], lw=1, color = 'k')
    
    
    point_tray = body_tray(X, params)
    point_tray_loop = loop_body_tray(X, params)
    #sys_cm_point, = ax.plot([], [], 'go', ms=12)
    #line_sys_cm, = ax.plot([], [], 'k:', lw=1)
    
    print_vars = [X[:,ii] for ii in range(5)]+[np.linspace(0, N-1, N, dtype=int)]
    print_var_names = [f'q_{ii}' for ii in range(5)]+['step']
    texts = []
    ii = 0.8
    for arr in print_vars:
        texts.append(ax.text(-0.8, ii, "", fontsize = 12))
        ii -= 0.2
    
    ax.grid()
    
    def init():
        body.set_data([], [])
        trail.set_data(point_tray_loop[0,0,-1], point_tray_loop[0,1,-1])
        old_trail.set_data(point_tray_loop[:,0,-1]-0.5, point_tray_loop[:,1,-1])
        #next_trail.set_data(point_tray_loop[:,0,-1]+0.5, point_tray_loop[:,1,-1])
        #sys_cm_point.set_data([], [])
        #line_sys_cm.set_data([], [])
        return (body,)
    
    def animate(i):
        margin_x = -0.25 + i * 0.5/N
        trail.set_data(point_tray_loop[0:i+1,0,-1], point_tray_loop[0:i+1,1,-1])
        #sys_cm_coords = sys_cm_np(X[i,:], params)
        #sys_cm_point.set_data(sys_cm_coords)
        #line_sys_cm.set_data([0, sys_cm_coords[0]], [0, sys_cm_coords[1]])
        
        ax.set_xlim(( -1+ margin_x, 1+ margin_x))
        points_x, points_y = point_tray[i,:,:]
        body.set_data(points_x, points_y) 
        
        for ii in range(len(texts)):
            text = texts[ii]
            name = print_var_names[ii]
            arr = print_vars[ii]
            text.set_position((-0.9 + margin_x, 1.7 - 0.05*ii))
            if name == 'step':
                text.set_text("$step$ = " + str(arr[i]))
            else:
                text.set_text("$" + name + "$ = %.3f" % arr[i])
        return (body,)
    iterable_frames = sum([[jj for jj in range(N)]for kk in range(n_loops)], start = [])
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=iterable_frames, interval=20, 
                               blit=True)
    return anim





In [ ]:
anim = create_anim(results['hs_mod_parab'][25]['x'][:-1,:],results['hs_mod_parab'][25]['u'], params, 20)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
f = r"biped_animation_hd.mp4" 
writervideo = animation.FFMpegWriter(fps=25//0.7) 
anim.save(f, writer=writervideo)

In [ ]:
def __create_anim_simp(X, U, params):
    [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
    ] = params
    
    N = X.shape[0]
    fig, ax = plt.subplots()
    
    y_scale = 1
    min_x_cart = np.min(X[:,0])
    max_x_cart = np.max(X[:,0])
    cart_displ = max_x_cart-min_x_cart
    size_x = 2*y_scale + cart_displ
    size_y = 2*y_scale
    draw_width = 14
    draw_height = draw_width / size_x * size_y
    
    x_0 = X[:,0]
    y_0 = np.zeros_like(x_0)
    x_1 = x_0 + l_3*np.sin(X[:,1])
    y_1 = y_0 - l_3*np.cos(X[:,1])
    
    x_cm = (m_0 * x_0 + m_2 * x_1)/(m_0 + m_2)
    y_cm = (m_0 * y_0 + m_2 * y_1)/(m_0 + m_2)

    fig.set_size_inches([draw_width,draw_height])
    ax.set_xlim(( min_x_cart-y_scale, max_x_cart+y_scale))
    ax.set_ylim(( -y_scale, y_scale))

    circle1 = plt.Circle((0, 0), l_3, color='b', ls = ":", fill=False)
    ax.add_artist(circle1)
    ax.plot([min_x_cart - l_3, max_x_cart + l_3], [0,0], 'k', lw=1, ls = ':')

    line1, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], 'k', lw=1, ls = ':')
    line_cm, = ax.plot([], [], 'g', lw=1, ls = ':')
    point0, = ax.plot([], [], marker='o', markersize=15, color="red")
    point1, = ax.plot([], [], marker='o', markersize=15, color="red")
    point_cm, = ax.plot([], [], marker='o', markersize=10, color="green")
    u_max = max(np.max(np.abs(U[:,0])),1e-15)
    arrow_w = 0.05*l_3
    arrow_l = 0.2*l_3
    u_arrow = patches.Arrow(0, 0, 0, -arrow_l, color = 'gray',width = arrow_w)
    ax.add_patch(u_arrow)
    
    print_vars = [X[:,0], X[:,1], U[:,0], U[:,1], np.linspace(0, N-1, N, dtype=int)]
    print_var_names = ['q_0', 'q_1', 'u_0', 'u_1', 'step']
    texts = []
    ii = 0
    for arr in print_vars:
        texts.append(ax.text(0.2, ii, "", fontsize = 12))
        ii -= 0.08*l_3
    
    def init():
        line1.set_data([], [])
        line3.set_data([], [])
        line_cm.set_data([], [])
        point1.set_data([], [])
        circle1.center = (0, 0)
        return (line1,)
    def animate(i):
        circle1.center = (x_0[i], y_0[i])
        point0.set_data(x_0[i], y_0[i])
        line1.set_data([x_0[i], x_1[i]], [y_0[i], y_1[i]])    
        point1.set_data(x_1[i], y_1[i])
        point_cm.set_data(x_cm[i], y_cm[i])
        line3.set_data(x_1[:i], y_1[:i])
        line_cm.set_data(x_cm[:i], y_cm[:i])
        trans = Affine2D()
        u_arrow._patch_transform = trans.scale(U[i,0] * arrow_l / u_max, arrow_w).translate(x_0[i],0)
        for ii in range(len(texts)):
            text = texts[ii]
            name = print_var_names[ii]
            arr = print_vars[ii]
            text.set_text("$" + name + "$ = %.6f" % arr[i])
        return (line1,u_arrow)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim